In [ ]:
import requests
from LocalRun.src.load_data import load_data

# Define the URL
url = "http://127.0.0.1:8080/predict"

data = load_data(local=True)
clean_data = data.dropna(axis=0, how='any')
clean_data = clean_data.iloc[0:10]

# Define the JSON payload
payload = {
    "columns": clean_data.columns.tolist(),
    "data": clean_data.to_numpy().tolist(),
}

# Define headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, json=payload, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())